In [30]:
import pandas as pd
import os
import numpy as np

In [6]:
os.getcwd()

'/home/victor/Zinc_Finger_FYP/lab_data_processing'

In [9]:
raw_data = pd.read_csv('Olga,students,Shayma_2025-05-02 14-59-05_CFX OPUS 384.csv', skiprows = 19)

In [10]:
raw_data.head()

,Well,Fluor,Target,Content,Sample,Cq,Starting Quantity (SQ)
0,A01,FAM,NaN,Unkn,NaN,NaN,NaN
1,A02,FAM,NaN,Unkn,NaN,NaN,NaN
2,A03,FAM,NaN,Unkn,NaN,NaN,NaN
3,A04,FAM,NaN,Unkn,NaN,NaN,NaN
4,A05,FAM,NaN,Unkn,NaN,NaN,NaN


In [ ]:


OC_IB = raw_data[raw_data["Well"].isin(["A07", "B07", "C07"])]
NC_IB = raw_data[raw_data["Well"].isin(["A08", "B08", "C08"])]
OW_IB = raw_data[raw_data["Well"].isin(["A09", "B09", "C09"])]
NW_IB = raw_data[raw_data["Well"].isin(["A10", "B10", "C10"])]

OC_LP = raw_data[raw_data["Well"].isin(["F07", "E07", "D07"])]
NC_LP = raw_data[raw_data["Well"].isin(["F08", "E08", "D08"])]
OW_LP = raw_data[raw_data["Well"].isin(["F09", "E09", "D09"])]
NW_LP = raw_data[raw_data["Well"].isin(["F10", "E10", "D10"])]

OC_V= raw_data[raw_data["Well"].isin(["G07", "H07", "I07"])]
NC_V = raw_data[raw_data["Well"].isin(["G08", "H08", "I08"])]
OW_V = raw_data[raw_data["Well"].isin(["G09", "H09", "I09"])]
NW_V = raw_data[raw_data["Well"].isin(["G10", "H10", "I10"])]

NC_HT = raw_data[raw_data["Well"].isin(["J07", "K07", "L07"])]
OC_HT = raw_data[raw_data["Well"].isin(["J08", "K08", "L08"])]
NW_HT = raw_data[raw_data["Well"].isin(["J09", "K09", "L09"])]
OW_HT = raw_data[raw_data["Well"].isin(["J10", "K10", "L10"])]


Control_C = raw_data[raw_data["Well"].isin(["A06", "B06", "C06","E06", "G05", 'H05',"I05", "J06", "K06", 'L06'])]
Control_W = raw_data[raw_data["Well"].isin(["A11", "B11", "C11","E11", "F11", "G11", 'H11',"I11", "J11", "K11", 'L11'])]

In [93]:
Control_W

,Well,Fluor,Target,Content,Sample,Cq,Starting Quantity (SQ)
10,A11,FAM,NaN,Unkn,NaN,27.740362,NaN
34,B11,FAM,NaN,Unkn,NaN,27.852969,NaN
58,C11,FAM,NaN,Unkn,NaN,26.711200,NaN
106,E11,FAM,NaN,Unkn,NaN,26.390818,NaN
130,F11,FAM,NaN,Unkn,NaN,26.584405,NaN
154,G11,FAM,NaN,Unkn,NaN,28.029951,NaN
178,H11,FAM,NaN,Unkn,NaN,28.233796,NaN
202,I11,FAM,NaN,Unkn,NaN,27.032662,NaN
226,J11,FAM,NaN,Unkn,NaN,28.077688,NaN
250,K11,FAM,NaN,Unkn,NaN,27.797658,NaN


In [ ]:
def ΔCt(df):
    #extract Cq for FAM and VIC for each gene where FAM is GOI expression and VIC is housekeeping gene 
    FAM_Cq = df[df['Fluor'] == 'FAM']["Cq"]
    VIC_Cq = df[df['Fluor'] == 'VIC']['Cq']

    #1.0 compute mean and standard deviation
    FAM_mean = FAM_Cq.mean()
    VIC_mean = VIC_Cq.mean()
    FAM_stdev= FAM_Cq.std()
    VIC_stdev= VIC_Cq.std()

    #2.0 Calculate the Delta Ct Value
    Ct_target = FAM_mean
    Ct_reference = VIC_mean
    Ct = Ct_target - Ct_reference

    #3.0 Standard dev of Delta Ct
    s = np.sqrt(FAM_stdev**2 + VIC_stdev**2)
    return (Ct, s)


In [ ]:
def ΔΔCt_pipeline(data, control):
    Ct , s = ΔCt(data)
    Ct_control, s_control = ΔCt(control)

    #4.0 Calculate ΔΔCt
    #print('Ct STUFF: ', Ct, Ct_control)
    ΔΔCt = Ct - Ct_control

    #5.0 Std of ΔΔCt is std dev of Ct

    #6.0 Incorporate ΔΔCt std into fold-difference
    pos = 2 **(-ΔΔCt + s)
    neg = 2 **(-ΔΔCt - s)

    #print(f'{ΔΔCt} +/- {s}')
    
    print(f'The test sample has between {neg} and {pos} the amount of target RNA as the control' )
    return(2**(-ΔΔCt))


In [81]:
ΔΔCt_pipeline(NC_V, Control_C)

The test sample has between 0.5289653199368942 and 1.6800621221354393 the amount of target RNA as the control


np.float64(0.9427059976202709)

In [91]:
#WRN = [OW_IB, NW_IB, OW_LP, NW_LP, OW_V, NW_V, OW_HT, NW_HT]
#CREBBP = [OC_IB, NC_IB, OC_LP, NC_LP, OC_V, NC_V, OC_HT, NC_HT]

WRN = [OW_V, NW_V, OW_HT, NW_HT]
CREBBP = [OC_V, NC_V, OC_HT, NC_HT]

CREBBP_Results = []
WRN_Results = []

for item in CREBBP:
    CREBBP_Results.append(ΔΔCt_pipeline(item, Control_C))
for item in WRN:
    WRN_Results.append(ΔΔCt_pipeline(item, Control_W))


The test sample has between 0.23993409813502167 and 0.7374447177977133 the amount of target RNA as the control
The test sample has between 0.5289653199368942 and 1.6800621221354393 the amount of target RNA as the control
The test sample has between 0.08535857469442999 and 184.80796373382836 the amount of target RNA as the control
The test sample has between 0.15323295930875994 and 13.881380441575521 the amount of target RNA as the control
The test sample has between 1.3584548225661504 and 4.060775930917566 the amount of target RNA as the control
The test sample has between 1.0489584402577292 and 3.1275796378955523 the amount of target RNA as the control
The test sample has between 1.8562138913689357 and 2.3573123028527756 the amount of target RNA as the control
The test sample has between 3.0286277972444657 and 5.165881237266361 the amount of target RNA as the control


In [89]:
print(CREBBP_Results)

[np.float64(0.42064014702501934), np.float64(0.9427059976202709), np.float64(3.9717684192937908), np.float64(1.4584529489679665)]


In [90]:
print(WRN_Results)

[np.float64(2.34869764906331), np.float64(1.811270564755236), np.float64(2.091811617438391), np.float64(3.9554432763533005)]
